In [4]:
import finlab
from finlab import data
import pandas as pd
import datetime as dt
import numpy as np
import math
from dotenv import load_dotenv
import os
import time
from finlab.backtest import sim
import scipy.stats as stats

def config():
    load_dotenv()


    
config()

finlab.login(str(os.getenv('FINLAB_API')))

輸入成功!


In [5]:
with data.universe(market='TSE_OTC',category='金融'): # 上市+上櫃
    financialColumns =data.get('etl:adj_close').columns
with data.universe(market='TSE_OTC'):
    allColumns =data.get('etl:adj_close').columns

with data.universe(market='TSE',category='金融'): # 上市 (ETF母體)
    financialTseColumns =data.get('etl:adj_close').columns
with data.universe(market='TSE'):
    allTseColumns =data.get('etl:adj_close').columns

# 上櫃
with data.universe(market='OTC',category='金融'):
    financialOtcColumns =data.get('etl:adj_close').columns
with data.universe(market='OTC'):
    allOtcColumns =data.get('etl:adj_close').columns

data.set_universe(market='TSE_OTC') #設定母體為TSE+OTC，為下一章的策略優化作準備


def Z(df):
    df= df.apply(stats.zscore,axis=1,nan_policy='omit') # zscore, 忽略nan
    cond = (df>3) | (df<-3) # 只留下3個std以內的資料,離群值不採納
    df[cond] = np.nan
    return df

def diffList(a,b): #在A裡面不在B裡面的元素
    return [i for i in a if i not in b]

Daily usage: 16.2 / 500 MB - etl:adj_close


In [10]:
收盤價 = data.get('etl:adj_close')
營業現金流量 = data.get('financial_statement:營業活動之淨現金流入_流出')
股本 = data.get('financial_statement:股本')
每股營業現金流量 = 營業現金流量 / (股本/10)
營業現金流量對股價比 = 每股營業現金流量 / 收盤價

# EBITDA對企業價值比
稅前息前折舊前淨利率 = data.get('fundamental_features:稅前息前折舊前淨利率')
營業收入淨額 = data.get("financial_statement:營業收入淨額")
EBITDA = 稅前息前折舊前淨利率*營業收入淨額/100
負債總額 = data.get('financial_statement:負債總額') 
現金及約當現金 = data.get('financial_statement:期末現金及約當現金餘額')
市值 = 股本/10*收盤價
企業價值 = 市值 + 負債總額 - 現金及約當現金
EBITDA對企業價值比 = EBITDA /企業價值

#wait some time otherwise there will be connection abort
time.sleep(5)



# 益本比
每股盈餘 = data.get('financial_statement:每股盈餘')
益本比 = 每股盈餘 / 收盤價

# 股東收益率
股息殖利率 = data.get('price_earning_ratio:殖利率(%)')
發行股數 = data.get('internal_equity_changes:發行股數') #改股本
前一年發行股數 = data.get('internal_equity_changes:發行股數').shift(12)  # 一年12個月
庫藏股回購率 = (發行股數 - 前一年發行股數) * (收盤價 / 市值)
股東收益率 = 股息殖利率 + 庫藏股回購率


Daily usage: 37.4 / 500 MB - internal_equity_changes:發行股數


In [11]:
# 計算價值因子分數

價值因子分數 = 0.4 * Z(營業現金流量對股價比) + 0.4* Z(益本比) + 0.2 * Z(股東收益率) #用非金融股算法計算全部
價值因子分數_金融 = 0.3 * Z(營業現金流量對股價比) +0.3 * Z(EBITDA對企業價值比) + 0.3 * Z(益本比) + 0.1 * Z(股東收益率) #用金融股算法計算全部
finCol = 價值因子分數_金融.columns.intersection(financialColumns) # 取交集過濾掉已經下市的股票
價值因子分數[finCol] = 價值因子分數_金融[finCol] # 再用金融股的值代入金融股的值
價值因子分數 # 所有TSE & OTC 的股票價值因子分數

symbol,1101,1102,1103,1104,1108,1109,1110,1201,1203,1210,...,9944,9945,9946,9949,9950,9951,9955,9958,9960,9962
date,,,,,,,,,,,,,,,,,,,,,
2013-04-19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-04-22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-04-23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-04-24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-04-25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-12-28,0.048464,0.075698,0.072644,0.048673,0.048029,0.121421,-0.012008,0.110650,0.023779,0.033412,...,0.087132,0.278542,-0.047253,-0.071388,-0.196733,0.046735,-0.064419,0.036663,0.041500,0.129412
2020-12-29,0.047778,0.073943,0.071465,0.049188,0.048283,0.120157,-0.013316,0.108292,0.023195,0.033119,...,0.085018,0.279620,-0.047801,-0.072350,-0.197551,0.045716,-0.065206,0.036229,0.032834,0.126635
2020-12-30,0.047426,0.074098,0.070649,0.048985,0.048265,0.120620,-0.013615,0.109965,0.023411,0.033602,...,0.084461,0.279360,-0.047836,-0.072497,-0.197506,0.047091,-0.065282,0.036198,0.033124,0.121934


In [ ]:
# 股東權益報酬率
# 每股盈餘 / 每股淨值。

股東權益報酬率 = data.get('fundamental_features:ROE稅後')
# 收益變動率

# 近五年每股盈餘成長率之樣本標準差取負值。
# III. 現金流量對資產比
# 營業現金流量 / 資產總額。
# IV. 資產報酬率
# 稅後純益 / 資產總額。
# V. 毛利對資產比
# 營業毛利 / 資產總額。
# VI. 槓桿度

In [ ]:
多因子分數 = 0.4 * 品質因子分數 + 0.3 * 價值因子分數 + 0.3 * 動能因子分數
多因子權重係數 = 多因子分數.applymap(lambda z: 1+z if(z>=0) else (1-z)**-1)